# 환경 설정

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

# Retrieval tool

## retrieval tool 1
- using PDFRetrievalChain

In [ ]:
from rag.pdf import PDFRetrievalChain

# PDF 문서를 로드합니다.
pdf = PDFRetrievalChain(["data/example.pdf"]).create_chain()

# retriever와 chain을 생성합니다.
pdf_retriever = pdf.retriever
pdf_chain = pdf.chain

In [ ]:
from langchain_core.tools.retriever import create_retriever_tool
from langchain_core.prompts import PromptTemplate

# PDF 문서를 기반으로 검색 도구 생성
retriever_tool = create_retriever_tool(
    pdf_retriever,
    "pdf_retriever",
    "use this tool to search information from the PDF document.",
    document_prompt=PromptTemplate.from_template(
        "<document><context>{page_content}</context><metadata><source>{source}</source><page>{page}</page></metadata></document>"
    ),
)

# 생성된 검색 도구를 도구 리스트에 추가하여 에이전트에서 사용 가능하도록 설정
tools = [retriever_tool]


## Retrieval 2
- using multi modal pkl data
- 사전에 이미 pdf를 pkl로 변환함

In [2]:
import pickle
import os

def load_documents_from_pkl(filepath):
    """
    Pickle 파일에서 Langchain Document 리스트를 불러오는 함수

    Args:
        filepath: 원본 파일 경로 (예: path/to/filename.pdf)
    Returns:
        Langchain Document 객체 리스트
    """
    # 확장자 제거하고 절대 경로로 변환
    abs_path = os.path.abspath(filepath)
    base_path = os.path.splitext(abs_path)[0]
    pkl_path = f"{base_path}.pkl"

    with open(pkl_path, "rb") as f:
        documents = pickle.load(f)
    return documents

In [5]:
import glob
from pathlib import Path

# extract_path 디렉토리에서 모든 .pkl 파일 찾기
extract_path = "data/84955c6c-9975-47cd-a4af-b9da78d548a5/"
pkl_files = glob.glob(str(Path(extract_path) / "**" / "*.pkl"), recursive=True)

if not pkl_files:
    print("❌ extract_path에서 .pkl 파일을 찾을 수 없습니다.")
else:
    # 모든 .pkl 파일에서 문서 로드
    all_documents = []
    for pkl_file in pkl_files:
        print(f"📄 {pkl_file} 파일 로드 중...")  # 한국어 코멘트
        documents = load_documents_from_pkl(pkl_file)
        all_documents.extend(documents)

    print(f"✅ 총 {len(all_documents)}개의 문서가 로드되었습니다.")

📄 data/84955c6c-9975-47cd-a4af-b9da78d548a5/84955c6c-9975-47cd-a4af-b9da78d548a5/84955c6c-9975-47cd-a4af-b9da78d548a5_RetirementPensionSubscriberTraining.pkl 파일 로드 중...
✅ 총 40개의 문서가 로드되었습니다.


In [6]:
# 문서 내용 확인
all_documents

[Document(metadata={'page': 0, 'source': '84955c6c-9975-47cd-a4af-b9da78d548a5_RetirementPensionSubscriberTraining.pdf'}, page_content='퇴직연금\n가입자 교육'),
 Document(metadata={'image': './images/figure/84955c6c-9975-47cd-a4af-b9da78d548a5_RetirementPensionSubscriberTraining_Page_0_Index_2.png)', 'entity': '<image>\n<title>\n경제 성장과 재정 관리의 시각화\n</title>\n<details>\n이미지는 경제 성장과 재정 관리를 상징적으로 표현하고 있으며, 사람들은 돈을 들고 있거나 쌓아놓고 있는 모습이 보입니다. 배경에는 도시 건물과 식물들이 있어 지속 가능한 발전을 암시합니다.\n</details>\n<entities>\n돈, 건물, 식물, 계산기\n</entities>\n<hypothetical_questions>\n- 현대 사회에서 재정 관리는 어떻게 변화하고 있을까?\n- 지속 가능한 경제 성장을 위해 개인이 할 수 있는 일은 무엇일까?\n</hypothetical_questions>\n</image>', 'page': 0, 'source': '84955c6c-9975-47cd-a4af-b9da78d548a5_RetirementPensionSubscriberTraining.pdf'}, page_content='![](./images/figure/84955c6c-9975-47cd-a4af-b9da78d548a5_RetirementPensionSubscriberTraining_Page_0_Index_2.png)\n\n<image>\n<title>\n경제 성장과 재정 관리의 시각화\n</title>\n<details>\n이미지는 경제 성장과 재정 관리를 상징적으로 표현하고 있으며, 사람들은 돈을 들고 있거나 쌓아

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools.retriever import create_retriever_tool

# VectorStore를 생성합니다.
vector = FAISS.from_documents(all_documents, OpenAIEmbeddings())

## 하이브리드 서치툴 생성

In [ ]:
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.retrievers import EnsembleRetriever

bm25_k = 4 # BM25 retriever의 k 파라미터 설정
semantic_k = 4 # semantic retriever의 k 파라미터 설정

docs = vector.docstore._dict.values()
texts = [doc.page_content for doc in docs]

# 키워드 서치
bm25_retriever = KiwiBM25Retriever.from_texts(texts)
bm25_retriever.k = bm25_k 

# 시멘틱 서치
semantic_retriever = vector.as_retriever( search_kwargs={"k": semantic_k} )

# 앙상블 리트리버 생성
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, semantic_retriever],
    weights=[0.5, 0.5],  # 각 리트리버의 가중치 설정
    search_kwargs={"k": bm25_k + semantic_k},  # 최종 검색 결과 개수
)

In [46]:
retriever_tool = create_retriever_tool(
    ensemble_retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

tools = [retriever_tool]

# langgraph

In [47]:
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages


# 에이전트 상태를 정의하는 타입 딕셔너리, 메시지 시퀀스를 관리하고 추가 동작 정의
class AgentState(TypedDict):
    # add_messages reducer 함수를 사용하여 메시지 시퀀스를 관리
    messages: Annotated[Sequence[BaseMessage], add_messages]


## node and edge

In [48]:
from typing import Literal
from langchain import hub
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import tools_condition

# 모델이름 가져오기
MODEL_NAME = "gpt-4o-mini"


# 데이터 모델 정의
class grade(BaseModel):
    """A binary score for relevance checks"""

    binary_score: str = Field(
        description="Response 'yes' if the document is relevant to the question or 'no' if it is not."
    )


def grade_documents(state) -> Literal["generate", "rewrite"]:
    # LLM 모델 초기화
    model = ChatOpenAI(temperature=0, model=MODEL_NAME, streaming=True)

    # 구조화된 출력을 위한 LLM 설정
    llm_with_tool = model.with_structured_output(grade)

    # 프롬프트 템플릿 정의
    prompt = PromptTemplate(
        template="""You are a grader assessing relevance of a retrieved document 
        to a user question. \n 
        Here is the retrieved document: \n\n {context} \n\n
        Here is the user question: {question} \n
        If the document contains keyword(s) or semantic meaning related to the 
        user question, grade it as relevant. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document 
        is relevant to the question.""",
        input_variables=["context", "question"],
    )

    # llm + tool 바인딩 체인 생성
    # | 체이닝 연산자
    # 입력을 받아서 프롬프트를 생성(프롬프트 템플릿) -> llm에서 구조화된 결과로 출력(llm_with_tool)
    # 추후 invoke() 메서드로 호출 실행
    chain = prompt | llm_with_tool

    # 현재 상태에서 메시지 추출
    messages = state["messages"]

    # 가장 마지막 메시지 추출
    last_message = messages[-1]

    # 원래 질문 추출
    question = messages[0].content

    # 검색된 문서 추출
    retrieved_docs = last_message.content

    # 관련성 평가 실행
    scored_result = chain.invoke({"question": question, "context": retrieved_docs})

    # 관련성 여부 추출
    score = scored_result.binary_score

    # 관련성 여부에 따른 결정
    if score == "yes":
        print("==== [DECISION: DOCS RELEVANT] ====")
        print(retrieved_docs)
        return "generate"

    else:
        print("==== [DECISION: DOCS NOT RELEVANT] ====")
        print(score)
        return "rewrite"


def agent(state):
    # 현재 상태에서 메시지 추출
    messages = state["messages"]

    # LLM 모델 초기화
    model = ChatOpenAI(temperature=0, streaming=True, model=MODEL_NAME)

    # retriever tool 바인딩
    model = model.bind_tools(tools)

    # 에이전트 응답 생성
    response = model.invoke(messages)

    # 기존 리스트에 추가되므로 리스트 형태로 반환
    return {"messages": [response]}


def rewrite(state):
    print("==== [QUERY REWRITE] ====")
    # 현재 상태에서 메시지 추출
    messages = state["messages"]
    # 원래 질문 추출
    question = messages[0].content

    # 질문 개선을 위한 프롬프트 구성
    msg = [
        HumanMessage(
            content=f""" \n 
    Look at the input and try to reason about the underlying semantic intent / 
    meaning. \n 
    Here is the initial question:
    \n ------- \n
    {question} 
    \n ------- \n
    Formulate an improved question: """,
        )
    ]

    # LLM 모델로 질문 개선
    model = ChatOpenAI(temperature=0, model=MODEL_NAME, streaming=True)
    # Query-Transform 체인 실행
    response = model.invoke(msg)

    # 재작성된 질문 반환
    return {"messages": [response]}


def generate(state):
    # 현재 상태에서 메시지 추출
    messages = state["messages"]

    # 원래 질문 추출
    question = messages[0].content

    # 가장 마지막 메시지 추출
    docs = messages[-1].content

    # RAG 프롬프트 템플릿 가져오기
    prompt = hub.pull("teddynote/rag-prompt")

    # LLM 모델 초기화
    llm = ChatOpenAI(model_name=MODEL_NAME, temperature=0, streaming=True)

    # RAG 체인 구성
    rag_chain = prompt | llm | StrOutputParser()

    # 답변 생성 실행
    response = rag_chain.invoke({"context": docs, "question": question})
    return {"messages": [response]}


## 그래프 생성

In [49]:
# LangGraph 라이브러리의 그래프 및 도구 노드 컴포넌트 임포트
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode

# AgentState 기반 상태 그래프 워크플로우 초기화
workflow = StateGraph(AgentState)

# 워크플로우 내 순환 노드 정의 및 추가
workflow.add_node("agent", agent)  # 에이전트 노드
retrieve = ToolNode([retriever_tool])
workflow.add_node("retrieve", retrieve)  # 검색 노드
workflow.add_node("rewrite", rewrite)  # 질문 재작성 노드
workflow.add_node("generate", generate)  # 관련 문서 확인 후 응답 생성 노드

# 시작점에서 에이전트 노드로 연결
workflow.add_edge(START, "agent")

# 검색 여부 결정을 위한 조건부 엣지 추가
workflow.add_conditional_edges(
    "agent",
    # 에이전트 결정 평가
    tools_condition,
    {
        # 조건 출력을 그래프 노드에 매핑
        "tools": "retrieve",
        END: END,
    },
)

# 액션 노드 실행 후 처리될 엣지 정의
workflow.add_conditional_edges(
    "retrieve",
    # 문서 품질 평가
    grade_documents,
)
workflow.add_edge("generate", END)
workflow.add_edge("rewrite", "agent")

# 워크플로우 그래프 컴파일
graph = workflow.compile()


In [50]:
from langgraph.graph.state import CompiledStateGraph
from langchain_core.runnables import RunnableConfig
from typing import Any, Dict, List, Callable

def stream_graph(
    graph: CompiledStateGraph,
    inputs: dict,
    config: RunnableConfig,
    node_names: List[str] = [],
    callback: Callable = None,
):
    """
    LangGraph의 실행 결과를 스트리밍하여 출력하는 함수입니다.

    Args:
        graph (CompiledStateGraph): 실행할 컴파일된 LangGraph 객체
        inputs (dict): 그래프에 전달할 입력값 딕셔너리
        config (RunnableConfig): 실행 설정
        node_names (List[str], optional): 출력할 노드 이름 목록. 기본값은 빈 리스트
        callback (Callable, optional): 각 청크 처리를 위한 콜백 함수. 기본값은 None
            콜백 함수는 {"node": str, "content": str} 형태의 딕셔너리를 인자로 받습니다.
    Returns:
        None: 함수는 스트리밍 결과를 출력만 하고 반환값은 없습니다.
    """
    prev_node = ""
    for chunk_msg, metadata in graph.stream(inputs, config, stream_mode="messages"):
        curr_node = metadata["langgraph_node"]

        # node_names가 비어있거나 현재 노드가 node_names에 있는 경우에만 처리
        if not node_names or curr_node in node_names:
            # 콜백 함수가 있는 경우 실행
            if callback:
                callback({"node": curr_node, "content": chunk_msg.content})
            # 콜백이 없는 경우 기본 출력
            else:
                # 노드가 변경된 경우에만 구분선 출력
                if curr_node != prev_node:
                    print("\n" + "=" * 50)
                    print(f"🔄 Node: \033[1;36m{curr_node}\033[0m 🔄")
                    print("- " * 25)
                print(chunk_msg.content, end="", flush=True)

            prev_node = curr_node

In [54]:
from langchain_core.runnables import RunnableConfig

# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": "0"})

# 사용자의 에이전트 메모리 유형에 대한 질문을 포함하는 입력 데이터 구조 정의
inputs = {
    "messages": [
        ("user", "퇴직연금 확정급여형(DB형)과 확정기여형(DC형)의 차이점은?"),
    ]
}

# 그래프 실행
stream_graph(graph, inputs, config, ["agent", "rewrite", "generate"])



🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
==== [DECISION: DOCS RELEVANT] ====
# PART 02 퇴직연금제도 일반
# 1. 퇴직연금제도의 이해
퇴직연금제도란?
• 회사(사용자)가 근로자의 퇴직급여 재원을 퇴직연금사업자(금융기관)에게 적립하고 사용자 또는 근로자의 지시에 따라
적립금이 운용되며 근로자 퇴직 시 연금 또는 일시금으로 퇴직급여를 지급하는 제도입니다.
• 퇴직연금제도를 도입할 경우 사용자가 근로자의 퇴직급여 지급을 위한 재원을 금융기관에 적립하기 때문에 기업이
도산 또는 파산하는 경우에도 근로자의 퇴직금은 안전하게 보장됩니다.
# 퇴직연금제도 종류
• 퇴직연금제도의 종류는 확정급여형퇴직연금(Defined Benefit), 확정기여형퇴직연금(Defined Contribution), 개인형
퇴직연금(Individual Retirement Pension)으로 구성됩니다.
DB
| 확정급여형(Defined Benefit)
- 회사가 쌓고 회사가 운용
- 사용자는 매년 부담금을 금융기관에 사외 적립하여 운용함
- 근로자는 정해진 퇴직급여 수령
DC
| 확정기여형퇴직연금(Defined Contribution)
- 회사는 가입자의 DC계좌로 부담금을 정기적으로 납입하고 가입자가 운용
- 근로자 본인이 직접 적립금에 대한 운용을 지시하므로 수익 또는 손실도 근로자에게 귀속
- 운용수익률 예상치가 급여상승률보다 높은 경우에 적합한 제도
| 개인형퇴직연금(Individual Retirement Pension)
- 근로자가 퇴직 시 퇴직급여가 입금되는 계좌
IRP
- 근로자가 이직, 퇴직으로 수령한 퇴직급여를 세제혜택(퇴직소득 과세이연)을 받으면서 은퇴 시점까지 계속 적립, 운용
하였다가 연금 또는 일시금으로 수령할 수 있는 계좌

| 구분 | 확정급여형(DB) | 확정기여형(DC)/ 기업형 IRP | 개인형 IRP |
| --- | --- | --- | --- |

/root/Project/aayn/.venv/lib/python3.10/site-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(



🔄 Node: generate 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
퇴직연금 확정급여형(DB형)과 확정기여형(DC형)의 차이점은 다음과 같습니다:

- **DB형 (확정급여형)**: 회사가 적립금을 쌓고 운용하며, 근로자는 정해진 퇴직급여를 수령합니다.
- **DC형 (확정기여형)**: 회사가 가입자의 DC계좌로 부담금을 정기적으로 납입하고, 가입자가 직접 운용하여 수익 또는 손실이 근로자에게 귀속됩니다.

**Source**
- (퇴직연금제도 일반, 페이지 1)

In [55]:
# 사용자의 에이전트 메모리 유형에 대한 질문을 포함하는 입력 데이터 구조 정의
inputs = {
    "messages": [
        ("user", "노후 대비를 위한 회사 제공 자산관리 옵션은 뭐가 있어?"),
    ]
}

# 그래프 실행
stream_graph(graph, inputs, config, ["agent", "rewrite", "generate"])


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
==== [DECISION: DOCS RELEVANT] ====
![](./images/figure/84955c6c-9975-47cd-a4af-b9da78d548a5_RetirementPensionSubscriberTraining_Page_2_Index_42.png)

<image>
<title>
재무 계획의 중요성
</title>
<details>
이미지는 재무 계획의 중요성을 강조하며, 자산을 키우기 위해 물을 주는 사람의 모습을 통해 자산 관리와 투자에 대한 비유를 나타냅니다.
</details>
<entities>
재무 계획, 자산, 투자, 물주기
</entities>
<hypothetical_questions>
- 어떻게 하면 효과적인 재무 계획을 세울 수 있을까?
- 자산을 키우기 위해 어떤 투자 전략이 가장 효과적일까?
</hypothetical_questions>
</image>

# 2. 노후준비 기본 전략
• 퇴직연금은 안정적인 노후를 위한 선진국형 3층 노후보장 체계에서 핵심적인 역할을 합니다. 국민연금으로 기초적인
생활을 보장받고, 퇴직연금으로 안정적인 노후 생활, 개인연금으로 여유있는 노후 생활을 준비할 수 있습니다.
• 퇴직연금은 노후의 안정적인 수입원 확보를 위한 최소한의 안전판이며 현재의 생활에 큰 불편을 주지 않으면서 노후를
대비할 수 있는 유일한 수단입니다.
3층(자기보장) - 여유로운 생활보장
개인연금
은퇴 후 여유로운 생활을 위해서 개인이 자발적으로 준비하는
연금제도
퇴직연금
2층(기업보장) - 안정적인 생활보장
국민연금
국민연금과는 별도로 안정적인 노후생활을 위해 노사협의에 의
해 자율적으로 가입하는 제도
1층(국가보장) - 기본적인생활보장
국민연금은 국민의 생활안정과 복지증진을 도모하기 위해
국가가 만든 사회보험제도
■ 한국의 노후보장은 3층 체계로서
1층 - 국민연금, 2층 - 퇴직연금, 3층 - 개

/root/Project/aayn/.venv/lib/python3.10/site-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(



🔄 Node: generate 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
회사가 제공하는 노후 대비 자산관리 옵션으로는 퇴직연금, 국민연금, 개인연금이 있습니다. 퇴직연금은 안정적인 노후 생활을 위한 핵심적인 수입원이며, 국민연금은 기본적인 생활 보장을, 개인연금은 여유로운 노후 생활을 준비하는 제도입니다.

**Source**
- 금융감독원 (페이지 미제공)